Let's collect data, transform a little and look at the description of it

In [7]:
import pandas as pd
import numpy as np

df = pd.read_excel(r'C:\Users\darar\Downloads\cereal.xlsx',header=None)
column_names = {
    0: 'cereal name',
	1: 'manufacturer',
	2: 'type',
	3: 'calories',
	4: 'protein',
	5: 'fat',
	6: 'sodium',
	7: 'dietary fiber',
	8: 'complex carbohydrates',
	9: 'sugars',
	10: 'display shelf',
	11: 'potassium',
	12: 'vitamins and minerals',
	13: 'weight',
	14: 'cups per serving' 
}
name_brands = {'A':'American Home Food Products', 'G':'General Mills', 'K':'Kelloggs', 'N':'Nabisco', 'P':'Post', 'Q':"Quaker Oats", 'R':'Ralston Purina'}

df[1] = df[1].fillna('')
df[2] = df[2].fillna('')
df[0] = df[0] + df[1] + df[2] 
df = df[0].str.split(' ', expand=True)
df.rename(columns={x: column_names[x] for x in column_names if x in df.columns}, inplace=True)
df['manufacturer'] = df['manufacturer'].map(name_brands)	
df = df.astype({x: 'float' for x in df.columns[3:]})
df.describe()


,calories,protein,fat,sodium,dietary fiber,complex carbohydrates,sugars,display shelf,potassium,vitamins and minerals,weight,cups per serving
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,106.883117,2.545455,1.012987,159.675325,2.151948,14.597403,6.922078,2.207792,96.077922,28.246753,0.977662,0.587273
std,19.484119,1.094790,1.006473,83.832295,2.383364,4.278956,4.444885,0.832524,71.286813,22.342523,0.358175,0.622659
min,50.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1.000000,-1.000000,0.000000,-1.000000,-1.000000
25%,100.000000,2.000000,0.000000,130.000000,1.000000,12.000000,3.000000,1.000000,40.000000,25.000000,1.000000,0.500000
50%,110.000000,3.000000,1.000000,180.000000,2.000000,14.000000,7.000000,2.000000,90.000000,25.000000,1.000000,0.750000
75%,110.000000,3.000000,2.000000,210.000000,3.000000,17.000000,11.000000,3.000000,120.000000,25.000000,1.000000,1.000000
max,160.000000,6.000000,5.000000,320.000000,14.000000,23.000000,15.000000,3.000000,330.000000,100.000000,1.500000,1.500000


Some MIN values in measures columns are negative. It can not be possible. Firstly change them to Nan 

In [8]:
col_selected_to_nan = [col for col in df.columns if df.dtypes[col] != 'object']
df.loc[:, col_selected_to_nan] = df.loc[:, col_selected_to_nan].applymap(lambda x: x if x >= 0 else np.nan)
df.loc[df.isna().values.any(axis=1),df.isna().values.any(axis=0)]

,complex carbohydrates,sugars,potassium,weight,cups per serving
1,8.0,8.0,135.0,1.00,NaN
4,14.0,8.0,NaN,1.00,0.75
20,21.0,0.0,NaN,1.00,1.00
38,17.0,6.0,60.0,1.00,NaN
43,16.0,3.0,95.0,1.00,NaN
44,16.0,11.0,170.0,NaN,NaN
45,16.0,11.0,170.0,NaN,NaN
50,18.0,2.0,90.0,1.00,NaN
55,10.0,0.0,50.0,0.50,NaN
57,NaN,NaN,110.0,1.00,0.67


We have to normalize data because of different weights, and cups per serving. However ['cups per serving'] has more missing values than ['weight']
For this: 
- delete column ['cups per serving'] because of too many NaN values
- delete rows with Nan values in ['weight'] (2 products only) beacause we don't know what the weight it can be  

In [9]:
df.drop(['cups per serving'], axis=1, inplace=True)
df.dropna(subset=['weight'], inplace=True)
df.loc[df.isna().values.any(axis=1),df.isna().values.any(axis=0)]

,complex carbohydrates,sugars,potassium
4,14.0,8.0,NaN
20,21.0,0.0,NaN
57,NaN,NaN,110.0


I suppose that it'll be a right way to fill remained NaN with a median across each column, where NaN value exists.  


In [10]:
df = df.fillna(df.median())

C:\Users\darar\AppData\Local\Temp\ipykernel_160\3493596106.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.median())


Now normalize all numeric data, except ['display shelf'], using ['weight] column

In [17]:
col_selected_to_normalize = [col for col in df.columns if df.dtypes[col] != 'object' and col != 'display shelf']
df.loc[:, col_selected_to_normalize] = df.loc[:, col_selected_to_normalize].div(df['weight'],axis=0)
df.drop(['weight'], axis=1, inplace=True)

Now let's put the dataframe into PowerQuery editor then build report in PowerBI

In [20]:
df

,cereal name,manufacturer,type,calories,protein,fat,sodium,dietary fiber,complex carbohydrates,sugars,display shelf,potassium,vitamins and minerals
0,100%_Bran,Nabisco,C,70.0,4.0,1.0,130.0,10.0,5.0,6.0,3.0,280.0,25.0
1,100%_Natural_Bran,Quaker Oats,C,120.0,3.0,5.0,15.0,2.0,8.0,8.0,3.0,135.0,0.0
2,All-Bran,Kelloggs,C,70.0,4.0,1.0,260.0,9.0,7.0,5.0,3.0,320.0,25.0
3,All-Bran_with_Extra_Fiber,Kelloggs,C,50.0,4.0,0.0,140.0,14.0,8.0,0.0,3.0,330.0,25.0
4,Almond_Delight,Ralston Purina,C,110.0,2.0,2.0,200.0,1.0,14.0,8.0,3.0,90.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,General Mills,C,110.0,2.0,1.0,250.0,0.0,21.0,3.0,3.0,60.0,25.0
73,Trix,General Mills,C,110.0,1.0,1.0,140.0,0.0,13.0,12.0,2.0,25.0,25.0
74,Wheat_Chex,Ralston Purina,C,100.0,3.0,1.0,230.0,3.0,17.0,3.0,1.0,115.0,25.0
75,Wheaties,General Mills,C,100.0,3.0,1.0,200.0,3.0,17.0,3.0,1.0,110.0,25.0
